### **ПРАКТИЧЕСКОЕ ЗАДАНИЕ**

**Задача: Поиск похожих по эмбеддингам.**

In [ ]:
# # Скачиваем датасет

# !wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz
# # Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

# # пример работы с ним 
# from corus import load_ods_gazeta
# path = 'gazeta.csv.gz'
# records = load_ods_gazeta(path)
# next(records)

На основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу).

Проверить насколько хорошо работают подходы.

In [2]:
# импортируем библиотеки

import json
import string
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
!pip install stop_words
from stop_words import get_stop_words
!pip install annoy
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
# дополнительные функции

# функция распоковки датасета со статьями
def get_datasets(path):
  data = []
  with open(path, 'r') as r:
      for line in r:
          data.append(json.loads(line))
  data.sort(key=lambda x: x['date'])
  return data


# функция извлечения названий и основного текста статей
def get_text(data):
  return [data['title'], data['text']]


# функция формирования токенов из текста
def get_tokens(data, stop_words=None, morph_analyzer=None):

  text = ''.join(i for i in data.strip() if i not in punct).split()  
  tokens = [tkn.lower() for tkn in text if tkn not in stop_words]
  tokens = [morpher.parse(tkn)[0].normal_form for tkn in tokens if len(tkn) > 1]
  return tokens


# функция возвращает 5 ближайших статей по запросу
def get_response(text, index, model, index_map):
    text = get_tokens(text)
    vector = np.zeros(300)
    norm = 0
    for word in text:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    titles = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in titles]


# функция возвращает текст по названию статьи
def read_article(article_name):
  for item in range(len(x_train)):
    if x_train[item]['title'] == article_name:
      return x_train[item]['text']

In [4]:
# загрузим статьи

!wget https://www.dropbox.com/s/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz
!tar -xvzf gazeta_jsonl_v2.tar.gz

--2022-08-27 08:50:25--  https://www.dropbox.com/s/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz [following]
--2022-08-27 08:50:25--  https://www.dropbox.com/s/raw/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc95d3a0ef964f5037ed968cdaa0.dl.dropboxusercontent.com/cd/0/inline/Brweqkm4Qno6mN5KrYt1ye0MuvKWMqxPUmSL883eqgQFyFwOOnbRbSJpp6WcClo4aqoBYS3P9ZYprf-179Ke9xSa_n3yfVPqzPLuaUNodHsfN6YAuYg2J8Xju8icUiPbEZaH1r4E9z0FA8HygaMfgzNxPGs0jho7jO9ivTGf6APOrg/file# [following]
--2022-08-27 08:50:25--  https://uc95d3a0ef964f5037ed968cdaa0.dl.dropboxusercontent.com/cd/0/inline/Brweqkm4Qno6mN5KrYt1ye0MuvKWMqxPUmS

In [5]:
# сформируем датасет

train_path = 'gazeta_train.jsonl'
# valid_path = 'gazeta_val.jsonl'
# test_path = 'gazeta_test.jsonl'

x_train = get_datasets(train_path)
# x_val = get_datasets(valid_path)
# x_test = get_datasets(test_path)

print(f'Train length: {len(x_train)}')
# print(f'Val length: {len(x_val)}')
# print(f'Test length: {len(x_test)}')

Train length: 60964


In [6]:
x_train[1]

{'date': '2010-06-01 10:42:59',
 'url': 'https://www.gazeta.ru/culture/2010/06/01/a_3378006.shtml',
 'summary': 'Британские затворники, московские модники, бразильский фанк и исламский панк, а также Мик Джаггер, группа Blur и фестивальные хроники, снятые на мобильник, – в Москве стартует Beat Film Festival.',
 'title': 'Секс, наркотики и темный зал',
 'text': "Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса. Так уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre. В столице картину показали год назад. Это был совсем камерный клубный показ в рамках фестиваля Music.doc, почти без рекламы, но о тех двух часах не пожалел никто из пары десятков случайных и не очень посетителей. То же можно сказать и о прочих показах. Стартующий 

In [7]:
x_train[1]['text']

"Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса. Так уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre. В столице картину показали год назад. Это был совсем камерный клубный показ в рамках фестиваля Music.doc, почти без рекламы, но о тех двух часах не пожалел никто из пары десятков случайных и не очень посетителей. То же можно сказать и о прочих показах. Стартующий в этот четверг Beat Film Festival – прямой наследник Music.doc. На этот раз к услугам киномеломанов самая авторитетная арт-хаусная площадка города, а не тесный клуб или помпезный «Иллюзион», а вместо собранной по сусекам программы – фильмы с не более чем двухлетним сроком давности. Картины «Bassweight» и «Favellla on Blast» рассказывают (соответственно) о все 

In [10]:
# извлечем названия и основной текст из статей

articles = []
for line in tqdm_notebook(x_train):
  articles.append(get_text(line))
  if len(articles) > 10000: # сделать на всем датасете
    break

  0%|          | 0/60964 [00:00<?, ?it/s]

In [12]:
articles[0]

['Налог в бак',
 'Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодн

In [13]:
# инициализируем морфоанализатор, запишем стоп-слова и пунктуацию в переменные

morpher = MorphAnalyzer()
stop_words = get_stop_words('ru')
punct = list(string.punctuation + '—' + '–')

In [15]:
# сформируем токены, соединив названия и основной текст статей в один список

tokens = []
for artcl in tqdm_notebook(articles):
  temp = []
  for item in artcl:    
    temp += get_tokens(item, stop_words, morpher)
  tokens.append(temp) 
  # if len(articles) > 2:
  #   break

  0%|          | 0/10001 [00:00<?, ?it/s]

In [18]:
tokens[140][:10]

['renaultnissan',
 'корейский',
 'кухня',
 'альянс',
 'renault',
 'nissan',
 'официально',
 'подтвердить',
 'заинтересованность',
 'приобретение']

In [19]:
# сформируем эмбеддинги статьей

modelW2V = Word2Vec(sentences=tokens, size=300, min_count=1, window=5)
modelFT = FastText(sentences=tokens, size=300, min_count=1, window=5)

In [23]:
print('музычка' in modelW2V.wv)
print('музычка' in modelFT.wv)

False
True


In [ ]:
# воспользуемся библиотекой Annoy, которая выполняет приближенный поиск, облегченный KNN

w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')
index_map = {}
counter = 0

for line in tqdm_notebook(articles):
        n_w2v = 0
        n_ft = 0  

        # записываем в словарь название статьи
        index_map[counter] = line[0]    

        # отделяем основной текст статьи
        article_body = get_tokens(line[1])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)        

        # идем по статьям и делаем эмбеддинги для токенов
        for word in article_body:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]                
                n_w2v += 1  # суммируем количество токенов в словаре
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        
        # усредняем, чтобы эмбеддинги не зависили от длины текста
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft

        # добавляем в индексы annoy
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 10000:
            break

# разбиваем на центроиды
w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/10001 [00:00<?, ?it/s]

True

### **РЕЗУЛЬТАТЫ**

In [ ]:
TEXT = 'транспорт'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

# Word2Vec - тематики выпали соответствующие запросу
# FastText - тематики выпали соответствующие запросу

['Светофоры поумнеют', 'Мэрия подавляет коллапс интеллектом ', 'Шереметьево поедет через область', 'Пробка подходит к развязке', 'Москва расколется по линии моста']
['Москва для пешеходов', 'Светофоры поумнеют', 'Фуры увязли в весне', 'Государство привьет транспорту интеллект', 'Иркутск не дотянул до нового аэропорта']


In [ ]:
TEXT = 'налоговый сбор'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

# Word2Vec - тематики выпали соответствующие запросу
# FastText - справился хуже, 2 раза выпала одна и та же статья

['Не как народ', 'Чиновников больше, зарплаты выше', 'Обама-писатель богаче Обамы-президента', 'Бюджет не справился с нулем', '«Для других аппетитов средств нет»']
['ОСАГО перевалило за кризис', 'Авиапассажиры пополнят казну Германии', 'Частичное экономическое затмение', 'Власть увлеклась кризисом', 'Авиапассажиры пополнят казну Германии']


In [ ]:
TEXT = 'искусственный интеллект'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

# Word2Vec - ожидалось, что выпадут статьи по теме IT, но названия больше соответвуют тематике "транспорт"
# FastText - выпали совершенно разные тематики

['Так делать дороги нельзя ни в коем случае', 'Porsche в зоне рулежки', 'Застывшее движение', 'Мал золотник, да дорог', 'Стамбул колоссальный']
['Manfrotto STILE — функциональность в итальянском стиле', 'Объем важен не только для прически', 'Нам нужны гаражи, а им – только наши деньги', 'Мал золотник, да дорог', 'Книги в интернете: скачать, прочесть, купить']


In [ ]:
TEXT = 'музыкальные новости'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

# Word2Vec - выпавшие тематики походят на запрос "футбол"
# FastText - только первая статья подходит под тематику

['Контадор ушёл из «Астаны»', '«Галатасарай» заинтересовался Андреем Аршавиным', 'Андрей в списках не значится', 'Сине-гранатовое доминирование ', 'Фабрика футбола']
['Албарн распустил Gorillaz', 'Губит людей не стаут', 'Королева встряхнула Англию', 'Камень с возу', '«Сплетница» созрела']


In [ ]:
TEXT = 'про футбол'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

# Word2Vec - тематики выпали соответствующие запросу
# FastText - тематики выпали соответствующие запросу

['«Думаю, в Катаре будет спокойно»', '«Мы бежим впереди паровоза»', '«Спаллетти может тренировать сборную»', '«Овечкин расколол НХЛ»', '«Любите своих футболистов»']
['«Селтик» без Макгиди', 'Невилл ушел на пенсию', 'Пареха вместе с Месси', 'Давидс вернулся в большой футбол', 'Жирков ждет Акинфеева']


In [ ]:
TEXT = 'как приготовить оливье'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

# Word2Vec - выпавшие тематики походят на запрос "мода"
# FastText - выпали совершенно разные тематики

['Миланская мода: в полоску и в клетку', 'Против яркого: модные тренды будущего сезона', 'Модно рвать и мять', 'Такая-сякая прикончила отца', 'Лондон: нефтяные пятна и голый живот']
['Сирота отстал под нож', 'Похитители приговорены к компенсации', 'Пираты осядут в Европе', 'В похищении чувствовалась женская рука', 'Стросс-Кан заинтересовал психиатров']


In [ ]:
read_article('Сирота отстал под нож')

'Убийство произошло в городе Сент-Франсисвилл в штате Луизиана. Преступление было совершено 10 июня около 13.00. 8-летний Кирилл Казаков , носивший после усыновления имя Джексон Аттусо, гулял с приемной матерью и братом в лесу, сообщает телевизионный канал Луизианы WAFB. Мальчик катался на велосипеде и отстал от своей семьи. В этот момент его увидел подросток — 16-летний местный житель Тревор Риз. Он подошел к ребенку и стал наносить ему удары ножом — в спину и грудь. После того как его жертва перестала сопротивляться, преступник перерезал мальчику горло. Окровавленного подростка, выходящего из леса, встретил местный житель. Как сообщает пресса, убийца сам попросил его набрать 911. «Мне позвонил плотник и сообщил, что из леса вышел парень весь в крови, который сказал, что убил другого ребенка, — рассказал суду шериф Остин Дэниэл. — Я тут же приехал на место. Я был просто в шоке от того, что увидел, это была ужасная сцена». Подозреваемый не мог объяснить полиции, за что он убил ребенка.

### **ВЫВОД**

В данной реализации лучше справляется метод **Word2Vec**, у него чаще выпадали ответы близкие по тематике запроса. Для повышения точности следует попробовать изменить некоторые параметры, например длину вектора эмбеддинга, количество центроид или добавить больше статей.